OCR WITH KERAS

In [13]:
import cv2
import os
import numpy as np
import imutils
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imutils import build_montages
from PIL import Image

from models import ResNet
import matplotlib
matplotlib.use("Agg")

EPOCHS = 50
INIT_LR = 1e-1
BS = 128

Datasets:

A-Z
http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/

Load A-Z dataset:

In [14]:
def load_a_z_font_dataset(rootdir):
    data = []
    labels = []
    
    for (dirpath, dirnames, filenames) in os.walk(rootdir):
        for filename in filenames:
            labels.append(int(filename[8:11])-11)
            
            image = cv2.imread(os.path.join(dirpath, filename))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            (thresh, bwImage) = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

            width, height = bwImage.shape
            totalpixels = width * height
            if cv2.countNonZero(bwImage) < totalpixels / 2:
                bwImage = np.invert(bwImage)

            (tH, tW) = bwImage.shape
            if tW > tH:
                bwImage = imutils.resize(bwImage, width=128)
            else:
                bwImage = imutils.resize(bwImage, height=128)

            (tH, tW) = bwImage.shape
            dX = int(max(0, 128 - tW) / 2.0)
            dY = int(max(0, 128 - tH) / 2.0)

            padded = cv2.copyMakeBorder(bwImage, top=dY, bottom=dY,
                left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
                value=(255, 255, 255))
                
            padded = cv2.resize(padded, (128, 128))
            padded = padded.astype("float32") / 255.0
            padded = np.expand_dims(padded, axis=-1)

            image = padded.reshape((128, 128))
            data.append(image)                    

    return (data, labels)

Pre-processing:

In [15]:
(data, labels) = load_a_z_font_dataset(r'C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\datasets\a_z_font')

data = [cv2.resize(image, (32, 32)) for image in data]
data = np.array(data, dtype="float32")

data = np.expand_dims(data, axis=-1)
data /= 255.0

le = LabelBinarizer()

labels = le.fit_transform(labels)
ounts = labels.sum(axis=0)

Training assesment balance:

In [16]:
classTotals = labels.sum(axis=0)
classWeight = {}

for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

(trainX, testX, trainY, testY) = train_test_split(data,
                                                  labels, test_size=0.20, stratify=None, random_state=42) #VER COMO USAR O STRATIFY

Preparing to train:

In [17]:
aug = ImageDataGenerator(rotation_range=10, zoom_range=0.05, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.15, horizontal_flip=False, fill_mode="nearest")

print("[INFO] compiling model...")

opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = ResNet.build(32, 32, 1, len(le.classes_), (3, 3, 3),
                     (64, 64, 128, 256), reg=0.0005)
                     
model.compile(loss="categorical_crossentropy",
              optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


C:\Users\alanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Training:

In [18]:
print("[INFO] training network...")

H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS), 
            validation_data=(testX, testY), 
            #steps_per_epoch=len(trainX),
            epochs=EPOCHS,
            class_weight=classWeight,
            verbose=1)

[INFO] training network...
Epoch 1/50
375/375 [==============================] - 544s 1s/step - loss: 4.1624 - accuracy: 0.2920 - val_loss: 2.0881 - val_accuracy: 0.5404
Epoch 2/50
375/375 [==============================] - 562s 1s/step - loss: 2.2030 - accuracy: 0.6396 - val_loss: 1.4067 - val_accuracy: 0.6800
Epoch 3/50
375/375 [==============================] - 531s 1s/step - loss: 1.5606 - accuracy: 0.7365 - val_loss: 1.1445 - val_accuracy: 0.7368
Epoch 4/50
375/375 [==============================] - 8775s 23s/step - loss: 1.3165 - accuracy: 0.7717 - val_loss: 1.0058 - val_accuracy: 0.7778
Epoch 5/50
135/375 [=========>....................] - ETA: 7:13:42 - loss: 1.2227 - accuracy: 0.7852

Network evaluation:

In [ ]:
labelNames = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
labelNames = [l for l in labelNames]

print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=labelNames))


[INFO] evaluating network...
              precision    recall  f1-score   support

           A       0.78      0.84      0.81       323
           B       0.87      0.85      0.86       211
           C       0.71      0.58      0.64       257
           D       0.86      0.86      0.86       235
           E       0.85      0.90      0.87       288
           F       0.92      0.89      0.91       236
           G       0.76      0.87      0.81       217
           H       0.80      0.87      0.83       254
           I       0.73      0.32      0.45       253
           J       0.86      0.82      0.84       216
           K       0.90      0.91      0.91       216
           L       0.88      0.87      0.88       242
           M       0.82      0.86      0.84       226
           N       0.86      0.85      0.86       274
           O       0.68      0.31      0.43       279
           P       0.81      0.86      0.84       212
           Q       0.74      0.86      0.80       20

Saving the model and training history:

In [ ]:
model_path = r"C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_full.model"
model.save(model_path, save_format="h5")

N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Trainning Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")


Testing:

In [ ]:
model_path = r"C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_full.model"
model = load_model(model_path)

images = [] 
for i in np.random.choice(np.arange(0, len(testY)), size=(49,)):
    probs = model.predict(testX[np.newaxis, i])
    prediction = probs.argmax(axis=1)
    label = labelNames[prediction[0]]

    image = (testX[i] * 255).astype("uint8")
    color = (0, 255, 0)

    if prediction[0] != np.argmax(testY[i]):
        color = (0, 0, 255)

    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)

    images.append(image)

montage = build_montages(images, (96, 96), (7, 7))[0]

cv2.imshow("OCR Results", montage)
cv2.waitKey(0)

-1

Predict:

In [ ]:
from tensorflow.keras.models import load_model
from imutils.contours import sort_contours
import numpy as np
import imutils
import cv2

model_path = r"C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_full.model"

print("[INFO] loading OCR model...")
model = load_model(model_path)
print(model_path)

image_path = r"C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\images\placa-perigo.png"

image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

edged = cv2.Canny(blurred, 30, 150)

cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sort_contours(cnts, method="left-to-right")[0]

chars = []

for c in cnts:
	(x, y, w, h) = cv2.boundingRect(c)

	if (w >= 20 and w <= 150) and (h >= 15 and h <= 120):
		roi = gray[y:y + h, x:x + w]
		thresh = cv2.threshold(roi, 0, 255,
			cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
		(tH, tW) = thresh.shape

		if tW > tH:
			thresh = imutils.resize(thresh, width=32)
		else:
			thresh = imutils.resize(thresh, height=32)

		(tH, tW) = thresh.shape
		dX = int(max(0, 32 - tW) / 2.0)
		dY = int(max(0, 32 - tH) / 2.0)

		padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,
			left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
			value=(255, 255, 255))
		
		padded = cv2.resize(padded, (32, 32))
		padded = padded.astype("float32") / 255.0
		padded = np.expand_dims(padded, axis=-1)

		chars.append((padded, (x, y, w, h)))

boxes = [b[1] for b in chars]
chars = np.array([c[0] for c in chars], dtype="float32")

preds = model.predict(chars)

labelNames = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
labelNames = [l for l in labelNames]

for (pred, (x, y, w, h)) in zip(preds, boxes):
	i = np.argmax(pred)
	prob = pred[i]
	if prob > 0.8:
		print("prob:",prob)
		label = labelNames[i]

		print("[INFO] {} - {:.2f}%".format(label, prob * 100))
		cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
		cv2.putText(image, label, (x - 10, y - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

cv2.imshow("Image", image)
cv2.waitKey(0)

[INFO] loading OCR model...
C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_only_capital.model
[(array([[[1.        ],
        [1.        ],
        [1.        ],
        ...,
        [1.        ],
        [1.        ],
        [1.        ]],

       [[1.        ],
        [1.        ],
        [1.        ],
        ...,
        [1.        ],
        [1.        ],
        [1.        ]],

       [[1.        ],
        [1.        ],
        [1.        ],
        ...,
        [1.        ],
        [1.        ],
        [1.        ]],

       ...,

       [[1.        ],
        [1.        ],
        [0.50980395],
        ...,
        [0.50980395],
        [1.        ],
        [1.        ]],

       [[1.        ],
        [1.        ],
        [0.09411765],
        ...,
        [0.07843138],
        [1.        ],
        [1.        ]],

       [[1.        ],
        [1.        ],
        [0.7764706 ],
        ...,
        [0.5568628 ],
        [1.    

-1